In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ----------------------
# 1. Define the Dataset
# ----------------------
class RegionDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        # Convert Region_ID to 0-indexed label
        label = int(row['Region_ID']) - 1  
        return image, label

# ----------------------
# 2. Define Transforms
# ----------------------
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ----------------------
# 3. Load CSV and Create Datasets
# ----------------------
# Set file paths for both training and validation
train_csv = "/kaggle/input/smainewdataset/Phase_2_data/labels_train.csv"
train_img_dir = "/kaggle/input/smainewdataset/Phase_2_data/images_train/images_train"
val_csv = "/kaggle/input/smainewdataset/Phase_2_data/labels_val.csv"
val_img_dir = "/kaggle/input/smainewdataset/Phase_2_data/images_val/images_val"

# Read CSV files
train_df = pd.read_csv(train_csv)
train_df['Region_ID'] = train_df['Region_ID'].astype(int)
val_df = pd.read_csv(val_csv)
val_df['Region_ID'] = val_df['Region_ID'].astype(int)
def filter_geo(df):
    return df[(df.latitude.between(200000,230000)) & (df.longitude.between(140000,150000))].copy()

train_df = filter_geo(train_df)
print("Train set size:", len(train_df))
print("External validation set size:", len(val_df))

# Create dataset objects
train_dataset = RegionDataset(train_df, train_img_dir, transform=train_transform)
val_dataset = RegionDataset(val_df, val_img_dir, transform=val_transform)

# Data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ----------------------
# 4. Model Builder Function
# ----------------------
def build_model(name='efficientnet_b0', num_classes=15, pretrained=True):
    if name == 'efficientnet_b0':
        model = models.efficientnet_b0(pretrained=pretrained)
        in_feats = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_feats, num_classes)
    elif name == 'resnet50':
        model = models.resnet50(pretrained=pretrained)
        in_feats = model.fc.in_features
        model.fc = nn.Linear(in_feats, num_classes)
    elif name == 'convnext_tiny':
        model = models.convnext_tiny(pretrained=pretrained)
        in_feats = model.classifier[2].in_features
        model.classifier[2] = nn.Linear(in_feats, num_classes)
    elif name == 'mobilenet_v3_large':
        model = models.mobilenet_v3_large(pretrained=pretrained)
        in_feats = model.classifier[3].in_features
        model.classifier[3] = nn.Linear(in_feats, num_classes)
    else:
        raise ValueError("Unknown model name")
    return model

# ----------------------
# 5. Training and Validation with Prediction Saving
# ----------------------
def train_validate(model, train_loader, val_loader, epochs=10, model_name='model'):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    model.to(device)
    
    best_acc = 0.0
    for epoch in range(epochs):
        # Training loop
        model.train()
        total_loss = 0.0
        train_pbar = tqdm(train_loader, desc=f"[Train Epoch {epoch+1}]", leave=False)
        for imgs, labels in train_pbar:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * imgs.size(0)
            train_pbar.set_postfix({"loss": loss.item()})
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        all_preds = []
        all_image_ids = []
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f"[Val Epoch {epoch+1}]", leave=False)
            for i, (imgs, labels) in enumerate(val_pbar):
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                preds = outputs.argmax(1)
                
                # Track image IDs for the validation set
                batch_indices = list(range(i * batch_size, min((i + 1) * batch_size, len(val_dataset))))
                all_image_ids.extend(batch_indices)
                all_preds.extend(preds.cpu().numpy())
                
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        acc = correct / total
        scheduler.step()
        print(f"Epoch {epoch+1}/{epochs}: Validation Accuracy: {acc:.6f}")
        
        # Save model and predictions if accuracy improves
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), f"best_{model_name}.pth")
            print(f"Model saved for {model_name} at epoch {epoch+1} (Acc: {acc:.6f})")

            # Save predictions to CSV
            # Get filenames from validation dataframe
            filenames = val_df['filename'].tolist()
            preds_df = pd.DataFrame({
                "filename": filenames[:len(all_preds)],
                "Region_ID": [p + 1 for p in all_preds]  # Convert to 1-indexed
            })
            acc_str = f"{acc:.4f}".replace(".", "_")  # Filename-safe
            preds_df.to_csv(f"Region_prediction_{acc_str}.csv", index=False)
            print(f"Predictions saved to Region_prediction_{acc_str}.csv")

    return best_acc

# ----------------------
# 6. Main: Build, Train, and Evaluate the Model
# ----------------------
model_name = 'convnext_tiny'  # Change to 'convnext_tiny', 'resnet50', etc.
model = build_model(name=model_name, num_classes=15, pretrained=True)
print("Training with model:", model_name)

# Train and evaluate
best_accuracy = train_validate(model, train_loader, val_loader, epochs=300, model_name=model_name)
print(f"Best Validation Accuracy for {model_name}: {best_accuracy:.6f}")

In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torchvision.models as models
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# from PIL import Image
# import pandas as pd
# from tqdm.notebook import tqdm
# from torch.optim.lr_scheduler import CosineAnnealingLR

# # ----------------------
# # 0. Device Configuration
# # ----------------------
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}\n")

# # ----------------------
# # 1. Dataset Class
# # ----------------------
# class RegionDataset(Dataset):
#     def __init__(self, dataframe, img_dir, transform=None):
#         self.df = dataframe
#         self.img_dir = img_dir
#         self.transform = transform
#         self.label_map = {label: idx for idx, label in enumerate(sorted(dataframe['Region_ID'].unique()))}

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         img_path = os.path.join(self.img_dir, row['filename'])
#         image = Image.open(img_path).convert("RGB")
        
#         if self.transform:
#             image = self.transform(image)
            
#         label = self.label_map[row['Region_ID']]
#         return image, label

# # ----------------------
# # 2. Transforms
# # ----------------------
# train_transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomVerticalFlip(p=0.2),
#     transforms.RandomRotation(15),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     transforms.RandomErasing(p=0.1, scale=(0.02, 0.1))
# ])

# val_transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# # ----------------------
# # 3. Data Loading
# # ----------------------
# def load_data(train_csv, val_csv, train_img_dir, val_img_dir):
#     # Load and filter dataframes
#     train_df = pd.read_csv(train_csv)
#     val_df = pd.read_csv(val_csv)
    
#     geo_filter = lambda df: df[(df.latitude.between(200000,230000)) & 
#                                (df.longitude.between(140000,150000))]
#     train_df = geo_filter(train_df)
    
#     print(f"Train samples: {len(train_df):,}")
#     print(f"Validation samples: {len(val_df):,}\n")
    
#     return (
#         RegionDataset(train_df, train_img_dir, train_transform),
#         RegionDataset(val_df, val_img_dir, val_transform)
#     )

# # ----------------------
# # 4. Model Builder
# # ----------------------
# def build_model(model_name='convnext_tiny', num_classes=15):
#     model_constructors = {
#         'convnext_tiny': models.convnext_tiny,
#         'efficientnet_b0': models.efficientnet_b0,
#         'resnet50': models.resnet50,
#         'mobilenet_v3_large': models.mobilenet_v3_large
#     }
    
#     try:
#         model = model_constructors[model_name](pretrained=True)
#     except KeyError:
#         raise ValueError(f"Unsupported model: {model_name}")

#     # Modify classifier
#     if 'convnext' in model_name:
#         in_features = model.classifier[-1].in_features
#         model.classifier[-1] = nn.Linear(in_features, num_classes)
#     elif 'efficientnet' in model_name:
#         in_features = model.classifier[-1].in_features
#         model.classifier[-1] = nn.Linear(in_features, num_classes)
#     elif 'resnet' in model_name:
#         in_features = model.fc.in_features
#         model.fc = nn.Linear(in_features, num_classes)
#     elif 'mobilenet' in model_name:
#         in_features = model.classifier[-1].in_features
#         model.classifier[-1] = nn.Linear(in_features, num_classes)
    
#     return model

# # ----------------------
# # 5. Training Engine
# # ----------------------
# class TrainingEngine:
#     def __init__(self, model, train_loader, val_loader, model_name):
#         self.model = model.to(device)
#         self.train_loader = train_loader
#         self.val_loader = val_loader
#         self.model_name = model_name
#         self.criterion = nn.CrossEntropyLoss()
#         self.current_epoch = 1
#         self.best_acc = 0.0

#         # Optimizer with differential learning rates
#         backbone_params = []
#         head_params = []
#         for name, param in model.named_parameters():
#             if any(k in name for k in ['classifier', 'fc', 'head']):
#                 head_params.append(param)
#             else:
#                 backbone_params.append(param)
                
#         self.optimizer = torch.optim.AdamW(
#             [
#                 {'params': backbone_params, 'lr': 1e-5},
#                 {'params': head_params, 'lr': 5e-5}
#             ],
#             weight_decay=1e-3
#         )
        
#         self.scheduler = CosineAnnealingLR(self.optimizer, T_max=50, eta_min=1e-7)

#     def load_checkpoint(self, checkpoint_path):
#         if checkpoint_path and os.path.exists(checkpoint_path):
#             print(f"\nLoading checkpoint: {checkpoint_path}")
#             ckpt = torch.load(checkpoint_path, map_location=device)
            
#             if isinstance(ckpt, dict) and 'model_state_dict' in ckpt:
#                 self.model.load_state_dict(ckpt['model_state_dict'], strict=False)
#                 self.best_acc = ckpt.get('best_acc', 0.0)
#                 self.current_epoch = ckpt.get('epoch', self.current_epoch - 1) + 1
                
#                 if 'optimizer_state_dict' in ckpt:
#                     self.optimizer.load_state_dict(ckpt['optimizer_state_dict'])
#                 if 'scheduler_state_dict' in ckpt:
#                     self.scheduler.load_state_dict(ckpt['scheduler_state_dict'])
                
#                 print(f"Resumed from epoch {self.current_epoch} | Best acc: {self.best_acc:.4f}")
#             else:
#                 self.model.load_state_dict(ckpt, strict=False)
#                 print("Loaded model weights only")
#         else:
#             print("\nNo valid checkpoint found, starting from scratch")

#     def _train_epoch(self, epoch):
#         self.model.train()
#         total_loss = 0.0
#         pbar = tqdm(self.train_loader, desc=f"Epoch {epoch} [Train]", leave=False)
        
#         for images, labels in pbar:
#             images, labels = images.to(device), labels.to(device)
            
#             self.optimizer.zero_grad()
#             outputs = self.model(images)
#             loss = self.criterion(outputs, labels)
#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
#             self.optimizer.step()
            
#             total_loss += loss.item()
#             pbar.set_postfix({'loss': f"{loss.item():.4f}"})
        
#         self.scheduler.step()
#         return total_loss / len(self.train_loader)

#     def _validate(self, epoch):
#         self.model.eval()
#         correct = 0
#         total = 0
#         all_preds = []
#         all_labels = []
        
#         with torch.no_grad():
#             pbar = tqdm(self.val_loader, desc=f"Epoch {epoch} [Val]", leave=False)
#             for images, labels in pbar:
#                 images = images.to(device)
#                 outputs = self.model(images)
#                 _, predicted = torch.max(outputs.data, 1)
                
#                 all_preds.extend(predicted.cpu().numpy())
#                 all_labels.extend(labels.cpu().numpy())
#                 total += labels.size(0)
#                 correct += (predicted.cpu() == labels).sum().item()
        
#         acc = correct / total
#         return acc, all_preds, all_labels

#     def run(self, epochs, checkpoint_path=None):
#         if checkpoint_path:
#             self.load_checkpoint(checkpoint_path)
        
#         start_epoch = self.current_epoch
#         for epoch in range(start_epoch, epochs + 1):
#             train_loss = self._train_epoch(epoch)
#             val_acc, preds, labels = self._validate(epoch)
            
#             print(f"Epoch {epoch:03d} | Loss: {train_loss:.4f} | Acc: {val_acc:.4f} | LR: {self.optimizer.param_groups[0]['lr']:.2e}")
            
#             # Update best model
#             if val_acc > self.best_acc:
#                 self.best_acc = val_acc
#                 torch.save({
#                     'epoch': epoch,
#                     'model_state_dict': self.model.state_dict(),
#                     'optimizer_state_dict': self.optimizer.state_dict(),
#                     'scheduler_state_dict': self.scheduler.state_dict(),
#                     'best_acc': self.best_acc
#                 }, f"best_{self.model_name}.pth")
                
#                 # Save predictions
#                 pd.DataFrame({
#                     'filename': self.val_loader.dataset.df['filename'],
#                     'true_label': labels,
#                     'predicted': [p+1 for p in preds]
#                 }).to_csv(f"best_predictions_{val_acc:.4f}.csv", index=False)
            
#             # Save latest checkpoint
#             torch.save({
#                 'epoch': epoch,
#                 'model_state_dict': self.model.state_dict(),
#                 'optimizer_state_dict': self.optimizer.state_dict(),
#                 'scheduler_state_dict': self.scheduler.state_dict(),
#                 'best_acc': self.best_acc
#             }, "latest_checkpoint.pth")
            
#             self.current_epoch += 1

#         return self.best_acc

# # ----------------------
# # 6. Main Execution
# # ----------------------
# if __name__ == "__main__":
#     # Configuration
#     CONFIG = {
#         'model_name': 'convnext_tiny',
#         'num_classes': 15,
#         'epochs': 30,
#         'batch_size': 128,
#         'checkpoint_path': "/kaggle/input/region-predict-97.83/tensorflow2/default/1/region_pred_97_83.pth",
#         'data_paths': {
#             'train_csv': "/kaggle/input/smainewdataset/Phase_2_data/labels_train.csv",
#             'val_csv': "/kaggle/input/smainewdataset/Phase_2_data/labels_val.csv",
#             'train_img_dir': "/kaggle/input/smainewdataset/Phase_2_data/images_train/images_train",
#             'val_img_dir': "/kaggle/input/smainewdataset/Phase_2_data/images_val/images_val"
#         }
#     }

#     # Load data
#     train_dataset, val_dataset = load_data(
#         CONFIG['data_paths']['train_csv'],
#         CONFIG['data_paths']['val_csv'],
#         CONFIG['data_paths']['train_img_dir'],
#         CONFIG['data_paths']['val_img_dir']
#     )

#     # Create dataloaders
#     train_loader = DataLoader(
#         train_dataset,
#         batch_size=CONFIG['batch_size'],
#         shuffle=True,
#         num_workers=4,
#         pin_memory=True
#     )
    
#     val_loader = DataLoader(
#         val_dataset,
#         batch_size=CONFIG['batch_size'],
#         shuffle=False,
#         num_workers=4,
#         pin_memory=True
#     )

#     # Initialize model
#     model = build_model(
#         model_name=CONFIG['model_name'],
#         num_classes=CONFIG['num_classes']
#     )

#     # Create training engine
#     engine = TrainingEngine(
#         model=model,
#         train_loader=train_loader,
#         val_loader=val_loader,
#         model_name=CONFIG['model_name']
#     )

#     # Run training
#     final_acc = engine.run(
#         epochs=CONFIG['epochs'],
#         checkpoint_path=CONFIG['checkpoint_path']
#     )

#     print(f"\nTraining completed. Best validation accuracy: {final_acc:.4f}")